# Generate Orders from Positions

In [212]:
import pandas as pd
import numpy as np
import io

### Data

In [213]:
def read(s):
    df = pd.read_csv(io.StringIO(s), sep="|") # io.StringIO() here let's us treat the string s as though it were a .csv file, necessary for pandas
    df.rename(columns = lambda s : s.strip(), inplace=True) # anonymous function stripping whitespace
    for col in df.columns:
        if col in ["dt", "rt"]:
            df[col] = pd.to_datetime(df[col]).dt.tz_localize('UTC') # need to localize to UTC to compare with timestamp
        elif df.dtypes[col] == np.dtype("O"): #dtype("0") are non-numeric np objects
            df[col] = df[col].apply(lambda s: s.strip())
        else:
            pass
    return df

1. rt - reference time: when the strategy wants to achieve given target position
2. dt - decision time: when the strategy decided to achieve that position

In [214]:
# Target Positions
tpos = read("""
    rt                   | strategy | asset | dt                   | value
    2023-08-29 07:00:00z | s1       | PLN   | 2023-08-29 07:00:00z | 4e6
    2023-08-29 16:00:00z | s1       | CZK   | 2023-08-29 07:00:00z | -24e6
    2023-08-30 07:00:00z | s1       | PLN   | 2023-08-30 07:00:00z | 8e6
    2023-08-30 16:00:00z | s1       | CZK   | 2023-08-30 07:00:00z | -48e6
    2023-08-29 07:00:00z | s2       | EUR   | 2023-08-29 07:00:00z | 1.8e6
    2023-08-29 16:00:00z | s2       | AUD   | 2023-08-29 07:00:00z | -1.5e6
    2023-08-30 07:00:00z | s2       | EUR   | 2023-08-30 07:00:00z | 3.6e6
    2023-08-30 16:00:00z | s2       | AUD   | 2023-08-30 07:00:00z | -1e6
    2023-08-29 07:00:00z | s3       | CZK   | 2023-08-29 07:00:00z | 12e6
    2023-08-29 16:00:00z | s3       | CZK   | 2023-08-29 07:00:00z | 18e6
    2023-08-30 07:00:00z | s3       | CZK   | 2023-08-30 07:00:00z | 18e6
    2023-08-30 16:00:00z | s3       | CZK   | 2023-08-30 07:00:00z | 24e6
    """)

In [215]:
fx_rates = read("""
    rt                   | asset | value
    2023-08-29 07:00:00z | PLN   | 3.934
    2023-08-29 16:00:00z | PLN   | 3.924
    2023-08-30 07:00:00z | PLN   | 3.914
    2023-08-30 16:00:00z | PLN   | 3.904
    
    2023-08-29 07:00:00z | CZK   | 23.12
    2023-08-29 16:00:00z | CZK   | 23.08
    2023-08-30 07:00:00z | CZK   | 23.02
    2023-08-30 16:00:00z | CZK   | 23.01
    
    2023-08-29 07:00:00z | EUR   | 1.116
    2023-08-29 16:00:00z | EUR   | 1.119
    2023-08-30 07:00:00z | EUR   | 1.121
    2023-08-30 16:00:00z | EUR   | 1.122
    
    2023-08-29 07:00:00z | AUD   | 0.672
    2023-08-29 16:00:00z | AUD   | 0.682
    2023-08-30 07:00:00z | AUD   | 0.689
    2023-08-30 16:00:00z | AUD   | 0.690
    """)

In [216]:
min_order_size_usd = read("""
    asset | value
    PLN   | 5e5
    CZK   | 5e5
    EUR   | 1e6
    AUD   | 1e6
""")

In [217]:
trading_session = pd.Timestamp("2023-08-30 16:00:00z")

### Easy Task: Generate orders (total & by strategy) as of trading session in local currency

In [254]:
todays_tpos = tpos[tpos['rt'] == trading_session]
print("Assuming all positions are zero to begin with, today's trades are:")
print(todays_tpos[['strategy', 'asset', 'value']])

Assuming all positions are zero to begin with, today's trades are:
   strategy asset       value
3        s1   CZK -48000000.0
7        s2   AUD  -1000000.0
11       s3   CZK  24000000.0


### Medium Task: Generate orders (total & by strategy) as of trading session in USD

In [256]:
order_by_strategy_and_asset_usd = pd.DataFrame([
    {'strategy': 's1', 'asset': 'PLN', 'order_usd': 0.0},
    {'strategy': 's1', 'asset': 'CZK', 'order_usd': -1043024.7718383311},
    {'strategy': 's2', 'asset': 'EUR', 'order_usd': 0.0},
    {'strategy': 's2', 'asset': 'AUD', 'order_usd': 344999.99999999994},
    {'strategy': 's3', 'asset': 'CZK', 'order_usd': 260756.19295958278}
])
order_by_strategy_and_asset_usd

,strategy,asset,order_usd
0,s1,PLN,0.000000e+00
1,s1,CZK,-1.043025e+06
2,s2,EUR,0.000000e+00
3,s2,AUD,3.450000e+05
4,s3,CZK,2.607562e+05


In [257]:
order_by_asset_usd = pd.DataFrame([
    {'asset': 'AUD', 'order_usd': 344999.99999999994},
    {'asset': 'CZK', 'order_usd': -782268.5788787483},
    {'asset': 'EUR', 'order_usd': 0.0},
    {'asset': 'PLN', 'order_usd': 0.0}
])
order_by_asset_usd

,asset,order_usd
0,AUD,345000.000000
1,CZK,-782268.578879
2,EUR,0.000000
3,PLN,0.000000


# Hard Task: Apply Minimum Order Size and generate new target position after this session

*Context: In practice there is often minimum cost we need to pay when trading. This means the orders must be of certain size to make economic sense. Therefore, if order is below the limit size it will not be executed and this needs to be fed back to the target position, so that during next strategy it trades orders knowing that past position is as of T-2, not T-1 (since we skipped T-1 orders)*

In [258]:
""" Minimum absolute size of the order in USD, below which we do not trade """
min_order_size_usd

,asset,value
0,PLN,500000.0
1,CZK,500000.0
2,EUR,1000000.0
3,AUD,1000000.0


In [259]:
valid_orders_usd = pd.DataFrame([
    {'asset': 'AUD', 'order_usd': 344999.99999999994, 'valid_orders': 0.0},
    {'asset': 'CZK','order_usd': -782268.5788787483, 'valid_orders': -782268.5788787483},
    {'asset': 'EUR', 'order_usd': 0.0, 'valid_orders': 0.0},
    {'asset': 'PLN', 'order_usd': 0.0, 'valid_orders': 0.0}
])
valid_orders_usd

,asset,order_usd,valid_orders
0,AUD,345000.000000,0.000000
1,CZK,-782268.578879,-782268.578879
2,EUR,0.000000,0.000000
3,PLN,0.000000,0.000000


In [260]:
""" 
Here I find exactly which orders were scaled down and by what scalar, 
which I will need when calculating final orders by strategy and asset
"""
valid_orders_usd['fx_rate'] = valid_orders_usd['asset'].apply(
    lambda x: 1/fx_rates.loc[trading_session][x] if x in ['EUR', 'AUD'] else fx_rates.loc[trading_session][x]
)

valid_orders_usd['valid_orders_local'] = valid_orders_usd['valid_orders'] * valid_orders_usd['fx_rate']
valid_orders_usd['scalar'] = (valid_orders_usd['valid_orders'] / valid_orders_usd['order_usd']).fillna(1)
valid_orders_scalar = valid_orders_usd.set_index('asset')['scalar']
valid_orders_scalar

KeyError: Timestamp('2023-08-30 16:00:00+0000', tz='UTC')

In [129]:
o_sa_local = order_by_strategy_and_asset_local_ccy # for convenience
o_sa_local['order_local_ccy_validated'] = o_sa_local.apply(
    lambda row: valid_orders_scalar[row['asset']] * row['order_local_ccy'], axis=1
)
order_as_valid = o_sa_local.copy()
order_as_valid['dt'] = pd.Timestamp("2023-08-30 07:00:00z")
order_as_valid['rt'] = trading_session
order_as_valid

,strategy,asset,order_local_ccy,order_local_ccy_validated,dt,rt
0,s1,PLN,0.0,0.0,2023-08-30 07:00:00+00:00,2023-08-30 16:00:00+00:00
1,s1,CZK,-24000000.0,-24000000.0,2023-08-30 07:00:00+00:00,2023-08-30 16:00:00+00:00
2,s2,EUR,0.0,0.0,2023-08-30 07:00:00+00:00,2023-08-30 16:00:00+00:00
3,s2,AUD,500000.0,0.0,2023-08-30 07:00:00+00:00,2023-08-30 16:00:00+00:00
4,s3,CZK,6000000.0,6000000.0,2023-08-30 07:00:00+00:00,2023-08-30 16:00:00+00:00


In [130]:
""" Adding those orders back to positions to create final target position """

original_positions = tpos.unstack(['strategy', 'asset']).ffill()
original_positions_drop_last_row = original_positions[
    original_positions.index.get_level_values('rt') != trading_session]

clean_new_orders = order_as_valid.set_index(['rt', 'dt', 'strategy', 'asset'])['order_local_ccy_validated']

new_last_row = original_positions_drop_last_row.iloc[-1] + clean_new_orders

In [131]:
""" Final Result """
modified_position_final = pd.concat([original_positions_drop_last_row, new_last_row.unstack(['strategy', 'asset'])], axis=0)
modified_position_final

strategy                                                    s1              \
asset                                                      PLN         CZK   
rt                        dt                                                 
2023-08-29 07:00:00+00:00 2023-08-29 07:00:00+00:00  4000000.0         NaN   
2023-08-29 16:00:00+00:00 2023-08-29 07:00:00+00:00  4000000.0 -24000000.0   
2023-08-30 07:00:00+00:00 2023-08-30 07:00:00+00:00  8000000.0 -24000000.0   
2023-08-30 16:00:00+00:00 2023-08-30 07:00:00+00:00  8000000.0 -48000000.0   

strategy                                                    s2             \
asset                                                      EUR        AUD   
rt                        dt                                                
2023-08-29 07:00:00+00:00 2023-08-29 07:00:00+00:00  1800000.0        NaN   
2023-08-29 16:00:00+00:00 2023-08-29 07:00:00+00:00  1800000.0 -1500000.0   
2023-08-30 07:00:00+00:00 2023-08-30 07:00:00+00:00  3600000.0 -1500000.0   
2023-08-30 16:00:00+00:00 2023-08-30 07:00:00+00:00  3600000.0 -1500000.0   

strategy                                                     s3  
asset                                                       CZK  
rt                        dt                                     
2023-08-29 07:00:00+00:00 2023-08-29 07:00:00+00:00  12000000.0  
2023-08-29 16:00:00+00:00 2023-08-29 07:00:00+00:00  18000000.0  
2023-08-30 07:00:00+00:00 2023-08-30 07:00:00+00:00  18000000.0  
2023-08-30 16:00:00+00:00 2023-08-30 07:00:00+00:00  24000000.0